In [1]:
import requests

headers = {
    'accept': '*/*',
}

params = {
    'page': '1',
    'perPage': '10',
    'serviceKey': 'ruy8A/v/syRNrte7Dyj7znYCM1HtSahtkKc7ZHTQUu+9dzLXkgdmTkbKhSwUqNwY2ksAHgrKPg/eV2SGtl1N6A==',
}

response = requests.get(
    'https://api.odcloud.kr/api/15084167/v1/uddi:1c8b5454-bd4e-45db-98f7-fe94d71f271b',
    params=params,
    headers=headers,
)

In [2]:
response

<Response [200]>

In [4]:
response.json()

{'currentCount': 10,
 'data': [{'구분번호': 1310440,
   '글제목': '대통령 취임사(大統領就任辭)',
   '대통령': '이승만',
   '연설일자': '1948-07-24',
   '연설장소': '국내',
   '원문보기': 'http://dams.pa.go.kr:8888/dams/DOCUMENT/2009/11/26/DOC/SRC/0104200911263997500039975011942.PDF'},
  {'구분번호': 1310441,
   '글제목': '민족이 원하는 길을 따를 결심, 국무총리 인준 부결에 대하여',
   '대통령': '이승만',
   '연설일자': '1948-07-29',
   '연설장소': '국내',
   '원문보기': 'http://dams.pa.go.kr:8888/dams/DOCUMENT/2009/11/26/DOC/SRC/0104200911263997600039976011943.PDF'},
  {'구분번호': 1310442,
   '글제목': '미급점(未及點) 육성하라',
   '대통령': '이승만',
   '연설일자': '1948-08-09',
   '연설장소': '국내',
   '원문보기': 'http://dams.pa.go.kr:8888/dams/DOCUMENT/2009/11/26/DOC/SRC/0104200911263997700039977011944.PDF'},
  {'구분번호': 1310443,
   '글제목': '전민족에게 충고함',
   '대통령': '이승만',
   '연설일자': '1948-08-12',
   '연설장소': '국내',
   '원문보기': 'http://dams.pa.go.kr:8888/dams/DOCUMENT/2009/11/26/DOC/SRC/0104200911263997800039978011945.PDF'},
  {'구분번호': 1310445,
   '글제목': '함께 뭉쳐서 자강전진(自疆前進) 외모(外侮)막자 (해방 3주년 기념사)',
   '대통령': '이승만',

In [5]:
# https://stackoverflow.com/questions/45470964/python-extracting-text-from-webpage-pdf

In [60]:
import pandas as pd

# csv 파일 열기
df = pd.read_csv("president_speeches_sample100.csv")

In [61]:
df

,구분번호,대통령,글제목,연설일자,원문보기,연설장소
0,1310440,이승만,대통령 취임사(大統領就任辭),1948-07-24,http://dams.pa.go.kr:8888/dams/DOCUMENT/2009/1...,국내
1,1310441,이승만,"민족이 원하는 길을 따를 결심, 국무총리 인준 부결에 대하여",1948-07-29,http://dams.pa.go.kr:8888/dams/DOCUMENT/2009/1...,국내
2,1310442,이승만,미급점(未及點) 육성하라,1948-08-09,http://dams.pa.go.kr:8888/dams/DOCUMENT/2009/1...,국내
3,1310443,이승만,전민족에게 충고함,1948-08-12,http://dams.pa.go.kr:8888/dams/DOCUMENT/2009/1...,국내
4,1310445,이승만,함께 뭉쳐서 자강전진(自疆前進) 외모(外侮)막자 (해방 3주년 기념사),1948-08-15,http://dams.pa.go.kr:8888/dams/DOCUMENT/2009/1...,국내
...,...,...,...,...,...,...
95,1310536,이승만,토탄을 사용하라,1949-09-30,http://dams.pa.go.kr:8888/dams/DOCUMENT/2009/1...,국내
96,1310537,이승만,"공산당과 협의 불가, 인권보증에 결사투쟁",1949-10-07,http://dams.pa.go.kr:8888/dams/DOCUMENT/2009/1...,국내
97,1310538,이승만,한글사용에 관하여,1949-10-12,http://dams.pa.go.kr:8888/dams/DOCUMENT/2009/1...,국내
98,1310539,이승만,대한 적십자사 재조직 중앙대회에 전하는 훈사,1949-10-25,http://dams.pa.go.kr:8888/dams/DOCUMENT/2009/1...,국내


In [63]:
import sqlite3
from pypdf import PdfReader
import requests
import io

connection = sqlite3.connect("president_speeches.db")
cursor = connection.cursor()
cursor.execute("""
CREATE TABLE IF NOT EXISTS president_speeches (
division_number INT,
president       TEXT,
title           TEXT,
date            TEXT,
pdf_url         TEXT,
location        TEXT,
speech_text     TEXT);
""")

connection.commit()

# csv 파일의 각 행을 순서대로 읽어 옴
for idx, row in df.iterrows():

    # 구분번호, 대통령, 글제목, 연설일자, 원문보기, 연설장소 를 변수에 저장
    print(row)
    division_number = row['구분번호']
    president = row['대통령']
    title = row['글제목']
    date = row['연설일자']
    pdf_url = row['원문보기']
    location = row['연설장소']
    
    r = requests.get(pdf_url)
    f = io.BytesIO(r.content)
    
    reader = PdfReader(f)
    page = reader.pages[0]
    # 연설원문을 변수에 저장
    speech_text = page.extract_text().replace("\n", "")

    # 데이터베이스에 저장
    cursor.execute("""INSERT INTO president_speeches (
    division_number, 
    president, 
    title, 
    date, 
    pdf_url, 
    location, 
    speech_text) VALUES (?, ?, ?, ?, ?, ?, ?);""", 
                   (division_number, president, title, date, pdf_url, location, speech_text))
    connection.commit()

# 데이터베이스 연결 종료
connection.close()


구분번호                                              1310440
대통령                                                   이승만
글제목                                      대통령 취임사(大統領就任辭) 
연설일자                                           1948-07-24
원문보기    http://dams.pa.go.kr:8888/dams/DOCUMENT/2009/1...
연설장소                                                   국내
Name: 0, dtype: object
구분번호                                              1310441
대통령                                                   이승만
글제목                    민족이 원하는 길을 따를 결심, 국무총리 인준 부결에 대하여 
연설일자                                           1948-07-29
원문보기    http://dams.pa.go.kr:8888/dams/DOCUMENT/2009/1...
연설장소                                                   국내
Name: 1, dtype: object
구분번호                                              1310442
대통령                                                   이승만
글제목                                        미급점(未及點) 육성하라 
연설일자                                           1948-08-09
원문보기    http://dams.pa.go.

In [1]:
# 1330137,이명박,한·아세안 특별정상회의(제2session) 인사말,2009-06-02,http://dams.pa.go.kr:8888/dams/DOCUMENT/2013/11/12/DOC/SRC/0104201311128517100085171018979.PDF,국내
# 1330138,이명박,한·아세안 특별정상회의(제2session)모두 발언,2009-06-02,http://dams.pa.go.kr:8888/dams/DOCUMENT/2013/11/12/DOC/SRC/0104201311128517200085172018980.PDF,국내

In [2]:
import sqlite3
from pypdf import PdfReader
import requests
import io

In [13]:
pdf_url = "http://dams.pa.go.kr:8888/dams/DOCUMENT/2013/11/12/DOC/SRC/0104201311128517100085171018979.PDF"
pdf_url = "http://dams.pa.go.kr:8888/dams/DOCUMENT/2013/11/12/DOC/SRC/0104201311128517000085170018978.PDF"

In [14]:
r = requests.get(pdf_url)
f = io.BytesIO(r.content)


In [15]:
reader = PdfReader(f)
page = reader.pages[0]
# 연설원문을 변수에 저장
speech_text = page.extract_text().replace("\n", "")

In [16]:
speech_text

''

In [17]:
len(speech_text)

0

In [18]:
# https://www.pa.go.kr/research/contents/speech/index.jsp?spMode=view&catid=c_pa02062&artid=1401643
# https://www.pa.go.kr/research/contents/speech/index.jsp?spMode=view&catid=c_pa02062&artid=1400011
# https://www.pa.go.kr/research/contents/speech/index.jsp?spMode=view&catid=c_pa02062&artid=1310352

In [22]:
import requests
import bs4

def crawl_document(url):
    response = requests.get(url, verify=False)
    html = response.text
    soup = bs4.BeautifulSoup(html, "html.parser")

    title = soup.select_one(".article-header h1").text
    text = soup.select_one(".article-content").text

    return title, text


title, text = crawl_document("https://www.pa.go.kr/research/contents/speech/index.jsp?spMode=view&catid=c_pa02062&artid=1310352")

print(title)
print(text)

/Users/m2/code/edu/president-speech/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.pa.go.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


AttributeError: 'NoneType' object has no attribute 'text'

In [53]:
import requests
import bs4

url = "https://www.pa.go.kr/research/contents/speech/index.jsp?spMode=view&catid=c_pa02062&artid=1310352"

# url = "https://www.pa.go.kr/research/contents/speech/index.jsp?spMode=view&catid=c_pa02062&artid=14016412"
response = requests.get(url, verify=False)
html = response.text
soup = bs4.BeautifulSoup(html, "html.parser")

/Users/m2/code/edu/president-speech/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.pa.go.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [24]:
# title = soup.select_one(".article-header h1").text
# text = soup.select_one(".article-content").text
# 
# return title, text
# 
# 
# title, text = crawl_document("https://www.pa.go.kr/research/contents/speech/index.jsp?spMode=view&catid=c_pa02062&artid=1310352")
# 
# print(title)
# print(text)

In [54]:
subject = soup.select("td.subject")
contents = soup.select("td.content")

In [65]:
subject[0].text

'정부환도 평양탈환 경축 유엔군환영 국민대회 대통령치사 속기록'

In [68]:
contents[0].text

AttributeError: 'str' object has no attribute 'text'

In [108]:
table = soup.find("tbody")
tr = table.find_all("tr")

In [28]:
table

<tbody>
<tr>
<td class="subject" colspan="6">정부환도 평양탈환 경축 유엔군환영 국민대회 대통령치사 속기록</td>
</tr>
<tr>
<th scope="row">연설일자</th>
<td>1950.10.27</td>
<th scope="row">대통령</th>
<td>이승만</td>
<th scope="row">연설장소</th>
<td>국내</td>
</tr>
<tr>
<th scope="row">유형</th>
<td>기념사</td>
<th scope="row">출처</th>
<td colspan="3">
                            -
                            
	                            <span class="mo-hidden" style="float: right; margin-left: 5px;">
<a class="button size2 color1 invert" href="javascript:void(0);" onclick="getMobileView('https://dams.pa.go.kr/dams/DOCUMENT/2020/10/30/DOC/SRC/0104202010301014268891004013998.pdf');return false;" title="새창열림">
	                                    원문보기
	                                    <i aria-hidden="true" class="icon-board-more2 lt"></i>
</a>
</span>
</td>
</tr>
<tr>
<td class="content" colspan="6">議長, 유엔代表團여러분, 各國外交使節團 여러분 一般同胞兄弟姉妹여러분 內亂을지난뒤 이자리에서 여러同胞를 이렇게 만나게되는 나로서는 마음에 넘치는 感想을 무엇이라고 이야기하기어렵습니다. 모르고 모르되 모든 同胞 여기 모이신 同胞가 다 나와같은 

In [32]:
tr = table.find_all("tr")

In [33]:
tr[0]

<tr>
<td class="subject" colspan="6">정부환도 평양탈환 경축 유엔군환영 국민대회 대통령치사 속기록</td>
</tr>

In [35]:
tr[1]

<tr>
<th scope="row">연설일자</th>
<td>1950.10.27</td>
<th scope="row">대통령</th>
<td>이승만</td>
<th scope="row">연설장소</th>
<td>국내</td>
</tr>

In [36]:
tds = tr[1].find_all('td')

In [38]:
tds[0].text

'1950.10.27'

In [39]:
tds[1].text

'이승만'

In [40]:
tds[2].text

'국내'

In [41]:
tr[2].find_all('td')[0]

<td>기념사</td>

In [42]:
tr[2].find_all('td')[1]

<td colspan="3">
                            -
                            
	                            <span class="mo-hidden" style="float: right; margin-left: 5px;">
<a class="button size2 color1 invert" href="javascript:void(0);" onclick="getMobileView('https://dams.pa.go.kr/dams/DOCUMENT/2020/10/30/DOC/SRC/0104202010301014268891004013998.pdf');return false;" title="새창열림">
	                                    원문보기
	                                    <i aria-hidden="true" class="icon-board-more2 lt"></i>
</a>
</span>
</td>

In [46]:
# https://www.pa.go.kr/research/contents/speech/index.jsp?spMode=view&catid=c_pa02062&artid=1401691   2017.05.10
# https://www.pa.go.kr/research/contents/speech/index.jsp?spMode=view&catid=c_pa02062&artid=1401692   2017.05.12
# 
# https://www.pa.go.kr/research/contents/speech/index.jsp?spMode=view&catid=c_pa02062&artid=1401641   22.03.30
# 
# https://www.pa.go.kr/research/contents/speech/index.jsp?spMode=view&catid=c_pa02062&artid=1401691
# 
# https://www.pa.go.kr/research/contents/speech/index.jsp?spMode=view&catid=c_pa02062&artid=1401639
# https://www.pa.go.kr/research/contents/speech/index.jsp?spMode=view&catid=c_pa02062&artid=1401640
# https://www.pa.go.kr/research/contents/speech/index.jsp?spMode=view&catid=c_pa02062&artid=1401641


In [ ]:
# https://www.pa.go.kr/research/contents/speech/index.jsp?spMode=view&catid=c_pa02062&artid=1310440 이승만
# https://www.pa.go.kr/research/contents/speech/index.jsp?spMode=view&catid=c_pa02062&artid=1310437 윤보선
# https://www.pa.go.kr/research/contents/speech/index.jsp?spMode=view&catid=c_pa02062&artid=1305369 박정희
# https://www.pa.go.kr/research/contents/speech/index.jsp?spMode=view&catid=c_pa02062&artid=1306636 최규하
# https://www.pa.go.kr/research/contents/speech/index.jsp?spMode=view&catid=c_pa02062&artid=1306788 전두환
# https://www.pa.go.kr/research/contents/speech/index.jsp?spMode=view&catid=c_pa02062&artid=1307296 노태우
# https://www.pa.go.kr/research/contents/speech/index.jsp?spMode=view&catid=c_pa02062&artid=1307905 김영삼
# https://www.pa.go.kr/research/contents/speech/index.jsp?spMode=view&catid=c_pa02062&artid=1308526 김대중
# https://www.pa.go.kr/research/contents/speech/index.jsp?spMode=view&catid=c_pa02062&artid=1309348 노무현
# https://www.pa.go.kr/research/contents/speech/index.jsp?spMode=view&catid=c_pa02062&artid=1310127 이명박
# https://www.pa.go.kr/research/contents/speech/index.jsp?spMode=view&catid=c_pa02062&artid=1400001 박근혜
# https://www.pa.go.kr/research/contents/speech/index.jsp?spMode=view&catid=c_pa02062&artid=1401691 문제인
# https://www.pa.go.kr/research/contents/speech/index.jsp?spMode=view&catid=c_pa02062&artid=1401641
https://www.pa.go.kr/research/contents/speech/index.jsp?spMode=view&catid=c_pa02062&artid=1401543
https://www.pa.go.kr/research/contents/speech/index.jsp?spMode=view&catid=c_pa02062&artid=1401629
https://www.pa.go.kr/research/contents/speech/index.jsp?spMode=view&catid=c_pa02062&artid=1401630
https://www.pa.go.kr/research/contents/speech/index.jsp?spMode=view&catid=c_pa02062&artid=1401637
https://www.pa.go.kr/research/contents/speech/index.jsp?spMode=view&catid=c_pa02062&artid=1401640
https://www.pa.go.kr/research/contents/speech/index.jsp?spMode=view&catid=c_pa02062&artid=1401473
# 



In [117]:
# for i in range(1400001, 1401691 + 1):
batch_data = []
for i in range(1400001, 1400100 + 1):
    all = []
    url = f"https://www.pa.go.kr/research/contents/speech/index.jsp?spMode=view&catid=c_pa02062&artid={i}"

    # print(url)
    response = requests.get(url, verify=False)
    html = response.text
    soup = bs4.BeautifulSoup(html, "html.parser")
    
    subject = soup.select("td.subject")[0].text
    contents = soup.select("td.content")[0].text
    
    table = soup.find("tbody")
    tr = table.find_all("tr")
    tds = tr[1].find_all('td')
    
    dt = tds[0].text
    president = tds[1].text
    place = tds[2].text
    
    kind = tr[2].find_all('td')[0].text
    
    batch_data.append((i, subject, contents, dt, president, place, kind))
    
# print(batch_data)

    

/Users/m2/code/edu/president-speech/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.pa.go.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/m2/code/edu/president-speech/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.pa.go.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/m2/code/edu/president-speech/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.pa.go.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  w

In [118]:
batch_data[0][0]

1400001

In [119]:
batch_data[0][1]

'제18대 대통령 취임식'

In [120]:
batch_data[0][2]

'희망의 새 시대를 열겠습니다.\n \xa0 \n존경하는 국민여러분! 700만 해외동포 여러분!\n \xa0 \n저는 오늘 대한민국의 제18대 대통령에 취임하면서 희망의 새 시대를 열겠다는 각오로 이 자리에 섰습니다. \n \xa0 \n저에게 이런 막중한 시대적 소명을 맡겨주신 국민 여러분께 깊이 감사드리며, 이 자리에 참석해주신 이명박 대통령과 전직 대통령, 그리고 세계 각국의 경축사절과 내외 귀빈 여러분께도 감사드립니다. \n \xa0 \n저는 대한민국의 대통령으로서 국민 여러분의 뜻에 부응하여 경제부흥과 국민행복, 문화융성을 이뤄낼 것입니다.\n \xa0 \n부강하고, 국민 모두가 함께 행복한 대한민국을 만드는데 저의 모든 것을 바치겠습니다!\n \xa0 \n국민여러분!\n \xa0 \n오늘의 대한민국은 국민의 노력과 피와 땀으로 이룩된 것입니다.\n하면 된다는 국민들의 강한 의지와 저력이 산업화와 민주화를 동시에 이룬 위대한 성취의 역사를 만들었습니다. \n \xa0 \n한강의 기적으로 불리는 우리의 역사는 독일의 광산에서, 열사의 중동 사막에서, 밤새 불이 꺼지지 않은 공장과 연구실에서, 그리고 영하 수 십도의 최전방 전선에서 가족과 조국을 위해 헌신하신 위대한 우리 국민들이 계셔서 가능했습니다.\n \xa0 \n저는 오늘의 대한민국을 만드신 모든 우리 국민들께 진심으로 경의를 표합니다. \n \xa0 \n존경하는 국민 여러분!\n \xa0 \n격동의 현대사 속에서 수많은 고난과 역경을 극복해 온 우리 앞에 지금 글로벌 경제 위기와 북한의 핵무장 위협과 같은 안보위기가 이어지고 있습니다.\n \xa0 \n글로벌 금융위기 이후 자본주의 역시, 새로운 도전에 직면해 있습니다.\n \xa0 \n이번 도전은 과거와는 달리 우리가 스스로, 새로운 길을 개척해야만 극복해나갈 수 있습니다.\n\n새로운 길을 개척하는 것은 쉽지 않은 일입니다. 그러나 저는 우리 대한민국의 국민을 믿습니다. 역동적인 우리 국민의 강인함과 저력을 믿습니다.\n \xa0 \n이제 자랑스

In [121]:
batch_data[0][3]

'2013.02.24'

In [122]:
batch_data[0][4]

'박근혜'

In [123]:
batch_data[0][5]

'국내'

In [124]:
batch_data[0][6]

'취임사'

In [125]:
for i in range(1400001, 1401692, 100):
  print(i)

1400001
1400101
1400201
1400301
1400401
1400501
1400601
1400701
1400801
1400901
1401001
1401101
1401201
1401301
1401401
1401501
1401601
